In [28]:
from dagster_acled.acled_request_config import * 
from dagster_acled.secrets_config import *
import os

sm = SecretManager(region_name=os.environ['REGION_NAME'])
client = AcledClientConfig()
token = await client.get_oauth_manager().get_access_token()

username = client.get_oauth_manager().username
password = client.get_oauth_manager().password

In [38]:
async def test_token(username: str, password: str, date: str):
    """Minimal test: get token, make API call."""
    
    # Get token
    data = aiohttp.FormData()
    data.add_field('username', username)
    data.add_field('password', password)
    data.add_field('grant_type', 'password')
    data.add_field('client_id', 'acled')
    
    async with aiohttp.ClientSession() as session:
        # Get token
        async with session.post('https://acleddata.com/oauth/token', data=data) as resp:
            token = (await resp.json())['access_token']
            print(f"Token: {token[:20]}...")
        
        headers = {'Authorization': f'Bearer {token}'}
        async with session.get(f'https://acleddata.com/api/acled/read?limit=1000&iso=804&event_date={date}', headers=headers) as resp:
            data = await resp.json()
            return data

In [39]:
import asyncio
import polars as pl

data = await test_token(username, password, date = '2025-09-26')
df = pl.DataFrame(data['data'])

with pl.Config(tbl_rows=df.shape[0], tbl_cols=df.shape[1]): 
    print(pl.DataFrame(df.collect_schema()))

Token: eyJ0eXAiOiJKV1QiLCJh...
shape: (1, 31)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
│ eve ┆ eve ┆ yea ┆ tim ┆ dis ┆ eve ┆ sub ┆ act ┆ ass ┆ int ┆ act ┆ ass ┆ int ┆ int ┆ civ ┆ iso ┆ reg ┆ cou ┆ adm ┆ adm ┆ adm ┆ loc ┆ lat ┆ lon ┆ geo ┆ sou ┆ sou ┆ not ┆ fat ┆ tag ┆ tim │
│ nt_ ┆ nt_ ┆ r   ┆ e_p ┆ ord ┆ nt_ ┆ _ev ┆ or1 ┆ oc_ ┆ er1 ┆ or2 ┆ oc_ ┆ er2 ┆ era ┆ ili ┆ --- ┆ ion ┆ ntr ┆ in1 ┆ in2 ┆ in3 ┆ ati ┆ itu ┆ git ┆ _pr ┆ rce ┆ rce ┆ es  ┆ ali ┆ s   ┆ est │
│ id_ ┆ dat ┆ --- ┆ rec ┆ er_ ┆ typ ┆ ent ┆ --- ┆ act ┆ --- ┆ --- ┆ act ┆ --- ┆ cti ┆ an_ ┆ obj ┆ --- ┆ y   ┆ --- ┆ --- ┆ --- ┆ on  ┆ de  ┆ ude ┆ eci ┆ --- ┆ _sc ┆ --- ┆ tie ┆ --- ┆ amp │
│ cnt ┆ e   ┆ obj ┆ isi ┆ typ ┆ e   ┆ _ty ┆ obj ┆ or_ ┆ obj ┆ obj ┆ or_ ┆ obj ┆ on  ┆ tar ┆ ect ┆ obj ┆ --- ┆ obj ┆ obj ┆ obj ┆ --- ┆ --- ┆ --- ┆ sio ┆ obj ┆ ale ┆ obj ┆ s   ┆ obj ┆ --- │
│ y   ┆ --- ┆ 

In [61]:
import polars as pl
import json

# Get individual file paths from the glob pattern
test_Df = pl.scan_parquet(
        "s3://dagster-acled-bucket/acled/acled_daily_data/acled_daily_data/partition_2025-09-26.parquet"
    ).collect()

In [ ]:


async def test_cookie_auth(username: str, password: str):
    """Minimal test: login with cookies, make API call."""
    
    async with aiohttp.ClientSession() as session:
        login_data = {
            "name": username,
            "pass": password
        }
        
        async with session.post(
            'https://acleddata.com/user/login?_format=json',
            json=login_data,
            headers={'Content-Type': 'application/json'}
        ) as login_resp:
            login_result = await login_resp.json()
            print(f"Login response: {login_result}")
            print(f"Cookies after login: {len(session.cookie_jar)} cookies")
        
        async with session.get(
            'https://acleddata.com/api/acled/read?limit=1&event_date=2024-12-31'
        ) as api_resp:
            data = await api_resp.json()
            print(f"API response status: {api_resp.status}")
            return data

result = await test_cookie_auth(username=username,password=password)
print(result)

Login response: {'current_user': {'uid': '59469', 'name': 's.a.bojilov@umail.leidenuniv.nl'}, 'csrf_token': '3_d7dxkGreXNTYwht65QrU3SZFXGkjF0m0OwW5temUY', 'logout_token': 'Vd3lt9wRVESord-vBM0uqWrkcoDD-BGI2UB6a6nhasQ'}
Cookies after login: 1 cookies
API response status: 200
{'status': 200, 'success': True, 'count': 0, 'total_count': 0, 'messages': [], 'data': [], 'filename': 'results.json', 'data_query_restrictions': {'countries': [], 'event_types': [], 'regions': [], 'history': [], 'recency': [], 'date_recency': {'quantity': 12, 'unit': 'Months', 'description': '12 Months old', 'timestamp': 1726586733, 'date': '2024-09-17'}}}


In [1]:
import duckdb
import boto3
# Get AWS credentials
session = boto3.Session()
credentials = session.get_credentials()

conn = duckdb.connect()
conn.execute("INSTALL httpfs;")
conn.execute("LOAD httpfs;")

# Set credentials from boto3
conn.execute(f"SET s3_access_key_id='{credentials.access_key}';")
conn.execute(f"SET s3_secret_access_key='{credentials.secret_key}';")
conn.execute("SET s3_region='eu-north-1';")
if credentials.token:
    conn.execute(f"SET s3_session_token='{credentials.token}';")

In [1]:
import polars as pl
import duckdb
from datetime import datetime, timedelta

# Setup DuckDB
conn = duckdb.connect()
conn.execute("INSTALL httpfs; LOAD httpfs;")
conn.execute("CALL load_aws_credentials();")
conn.execute("SET s3_region='eu-north-1';")

# Define dates
end_date = datetime.now().date()
start_date = end_date - timedelta(days=365)

# Convert to ISO format strings
start_date_str = start_date.isoformat()
end_date_str = end_date.isoformat()

result = pl.from_arrow(
    conn.execute("""
        SELECT 
            disorder_type,
            event_type,
            sub_event_type,
            actor1,
            actor2,
            inter1,
            inter2,
            interaction,
            admin1,
            admin2, 
            admin3,
            CAST(latitude AS DOUBLE) as latitude,
            CAST(longitude AS DOUBLE) as longitude,
            CAST(fatalities AS INTEGER) as fatalities,
            CAST(event_date AS DATE) as event_date
        FROM read_parquet('s3://dagster-acled-bucket/acled/acled_daily_data/partition_*.parquet')
        WHERE CAST(event_date AS DATE) >= CAST(? AS DATE)
            AND CAST(event_date AS DATE) <= CAST(? AS DATE)
            AND CAST(fatalities AS INTEGER) IS NOT NULL
            AND CAST(fatalities AS INTEGER) > 0
            AND CAST(latitude AS DOUBLE) IS NOT NULL 
            AND CAST(longitude AS DOUBLE) IS NOT NULL
        ORDER BY event_date
    """, [start_date_str, end_date_str]).arrow()
)

print(f"Loaded {len(result):,} records")
print(result.head())

Loaded 10,317 records
shape: (5, 15)
┌───────────┬───────────┬───────────┬───────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ disorder_ ┆ event_typ ┆ sub_event ┆ actor1    ┆ … ┆ latitude ┆ longitude ┆ fatalitie ┆ event_dat │
│ type      ┆ e         ┆ _type     ┆ ---       ┆   ┆ ---      ┆ ---       ┆ s         ┆ e         │
│ ---       ┆ ---       ┆ ---       ┆ str       ┆   ┆ f64      ┆ f64       ┆ ---       ┆ ---       │
│ str       ┆ str       ┆ str       ┆           ┆   ┆          ┆           ┆ i32       ┆ date      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ Political ┆ Explosion ┆ Shelling/ ┆ Military  ┆ … ┆ 47.8479  ┆ 35.1602   ┆ 6         ┆ 2024-12-3 │
│ violence  ┆ s/Remote  ┆ artillery ┆ Forces of ┆   ┆          ┆           ┆           ┆ 0         │
│           ┆ violence  ┆ /missile  ┆ Ukraine   ┆   ┆          ┆           ┆           ┆           │
│           ┆           ┆ att…      ┆ (20…      ┆   ┆ 

In [19]:
from dagster_acled.resources.resources import ResourceConfig
from dagster_acled.secrets_config import SecretManager
import os
import pandas as pd

sm = SecretManager(region_name=os.environ['REGION_NAME'])

resource_config = ResourceConfig.from_secrets(sm=sm, s3_secret_name="acled_bucket", pg_secret_name="acled_postgres")
postgres = resource_config.postgres
conn = postgres.get_connection()

query = f"""
    SELECT 
        event_id_cnty,
        event_date,
        disorder_type,
        event_type,
        sub_event_type,
        actor1,
        actor2,
        inter1,
        inter2,
        interaction,
        admin1,
        admin2,
        admin3,
        latitude,
        longitude,
        fatalities
    FROM {postgres.table_name} 
    WHERE event_date >= %s 
        AND event_date <= %s
        AND latitude IS NOT NULL 
        AND longitude IS NOT NULL
    ORDER BY event_date
    """

recent_df = pd.read_sql_query(query, conn, params=[start_date, end_date])
print(recent_df.describe())

/var/folders/rs/lp_7j4vj5_s59_pr61yffxl80000gn/T/ipykernel_70380/519628330.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  recent_df = pd.read_sql_query(query, conn, params=[start_date, end_date])


           latitude     longitude    fatalities
count  54249.000000  54249.000000  54249.000000
mean      48.929933     35.495349      1.041457
std        1.708559      2.178294      5.222928
min       43.389000     22.175900      0.000000
25%       47.663900     33.870700      0.000000
50%       48.436600     35.823500      0.000000
75%       50.305600     37.440100      0.000000
max       52.341300     40.132200    213.000000


In [18]:
import polars as pl
from datetime import datetime, timedelta

# Define date range
end_date = datetime.now().date()
start_date = end_date - timedelta(days=365)

# Scan with lazy evaluation
result = (
    pl.scan_parquet(
        "s3://dagster-acled-bucket/acled/acled_daily_data/partition_*.parquet",
    )
    .select([
        pl.col("disorder_type"),
        pl.col("event_type"),
        pl.col("sub_event_type"),
        pl.col("actor1"),
        pl.col("actor2"),
        pl.col("inter1"),
        pl.col("inter2"),
        pl.col("interaction"),
        pl.col("admin1"),
        pl.col("admin2"),
        pl.col("admin3"), 
        pl.col('year').cast(pl.Int64),
        pl.col("latitude").cast(pl.Float64).alias("latitude"),
        pl.col("longitude").cast(pl.Float64).alias("longitude"),
        pl.col("fatalities").cast(pl.Int32).alias("fatalities"),
        pl.col("event_date").str.strptime(pl.Date, "%Y-%m-%d").alias("event_date"),
    ])
    .filter(
        (pl.col("event_date") >= start_date) &
        (pl.col("event_date") <= end_date) &
        (pl.col("fatalities").is_not_null()) &
        (pl.col("fatalities") > 0) &
        (pl.col("latitude").is_not_null()) &
        (pl.col("longitude").is_not_null())
    )
    .sort("event_date", "event_type")
    .collect(engine="streaming") 
)

print(f"Loaded {len(result):,} records")
print(result.head())

Loaded 10,317 records
shape: (5, 16)
┌───────────┬───────────┬───────────┬───────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ disorder_ ┆ event_typ ┆ sub_event ┆ actor1    ┆ … ┆ latitude ┆ longitude ┆ fatalitie ┆ event_dat │
│ type      ┆ e         ┆ _type     ┆ ---       ┆   ┆ ---      ┆ ---       ┆ s         ┆ e         │
│ ---       ┆ ---       ┆ ---       ┆ str       ┆   ┆ f64      ┆ f64       ┆ ---       ┆ ---       │
│ str       ┆ str       ┆ str       ┆           ┆   ┆          ┆           ┆ i32       ┆ date      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ Political ┆ Battles   ┆ Armed     ┆ Military  ┆ … ┆ 47.9836  ┆ 37.2823   ┆ 3         ┆ 2024-12-3 │
│ violence  ┆           ┆ clash     ┆ Forces of ┆   ┆          ┆           ┆           ┆ 0         │
│           ┆           ┆           ┆ Russia    ┆   ┆          ┆           ┆           ┆           │
│           ┆           ┆           ┆ (200…     ┆   ┆ 

In [22]:
dates = recent_df['event_date'].sort_values().value_counts()

In [23]:
dates

event_date
2025-07-14    278
2025-06-02    274
2025-07-15    269
2025-06-17    268
2025-04-22    266
             ... 
2025-02-20    135
2025-06-26    126
2025-01-02    118
2025-05-08    113
2025-04-20     80
Name: count, Length: 264, dtype: int64